In [2]:
import pandas as pd

import csv
import sqlalchemy as db
from sqlalchemy import create_engine, types

In [2]:
# read 300 000 rows of csv file
df = pd.read_csv("../data/data_raw.csv", nrows=300000, sep='\t', low_memory=False)

In [187]:
# shape
df.shape

(300000, 186)

In [190]:
df.head(3)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,jeunes pousses,NaN,NaN,NaN,NaN,NaN,NaN,endives,endives,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [191]:
# options to display all the columns
pd.options.display.min_rows = 186

In [192]:
# sum of NaN values on all the columns
df.isnull().sum().sort_values(ascending=False)

-montanic-acid_100g                        300000
-dihomo-gamma-linolenic-acid_100g          300000
no_nutriments                              300000
additives                                  300000
ingredients_from_palm_oil                  300000
ingredients_that_may_be_from_palm_oil      300000
-caproic-acid_100g                         300000
-caprylic-acid_100g                        300000
-capric-acid_100g                          300000
-myristic-acid_100g                        300000
-melissic-acid_100g                        300000
-gamma-linolenic-acid_100g                 300000
-elaidic-acid_100g                         300000
cities                                     300000
-erucic-acid_100g                          300000
-glucose_100g                              300000
-fructose_100g                             300000
casein_100g                                300000
beta-carotene_100g                         300000
chlorophyl_100g                            300000


In [3]:
# keep only columns where 60% or more valid data is available
clean_df=df.dropna(how='any',axis=1,thresh=df.shape[0]*0.5)

In [15]:
# shape
clean_df.shape

(300000, 38)

In [16]:
# sum of NaN values on 38 columns
clean_df.isnull().sum().sort_values(ascending=False)

brand_owner                                111961
nova_group                                  95680
main_category_en                            92700
main_category                               92700
categories_tags                             92700
categories_en                               92700
categories                                  92699
serving_quantity                            89384
serving_size                                89309
ingredients_that_may_be_from_palm_oil_n     81549
ingredients_from_palm_oil_n                 81549
additives_n                                 81549
ingredients_text                            81549
saturated-fat_100g                          51956
sugars_100g                                 36285
salt_100g                                   33178
sodium_100g                                 33178
energy-kcal_100g                            26956
proteins_100g                               26326
fat_100g                                    26238


In [4]:
# remove some columns
clean_df.drop(['code', 'url', 'last_modified_t', 'last_modified_datetime', 'created_datetime', 'created_t', 'creator', 'states_en', 'categories_en', 'countries_tags', 'countries_en', 'main_category_en', 'states', 'states_tags'], axis=1, inplace=True)

C:\Users\Adil\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [5]:
# replace NaN by 0
clean_df = clean_df.fillna(0)

In [6]:
# rename column energy-kcal_100g 
clean_df.rename(columns={'energy-kcal_100g':'energy_kcal_100g', 'saturated-fat_100g':'saturated_fat_100g', 'trans-fat_100g':'trans_fat_100g', 'nutrition-score-fr_100g':'nutrition_score_fr_100g'}, inplace=True)

In [21]:
clean_df.head(3)

,product_name,categories,categories_tags,countries,ingredients_text,serving_size,serving_quantity,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,...,main_category,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g
0,jeunes pousses,NaN,NaN,en:france,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L.casei,NaN,NaN,Spain,"Leche semidesnatada, azucar 6.9% leche desnata...",NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,1.4,0.90,9.8,9.8,2.7,0.1,0.04
2,Vitória crackers,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,375.0,1569.0,7.0,3.08,70.1,15.0,7.8,1.4,0.56


In [22]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 24 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   product_name                             294390 non-null  object 
 1   categories                               207301 non-null  object 
 2   categories_tags                          207300 non-null  object 
 3   countries                                299512 non-null  object 
 4   ingredients_text                         218451 non-null  object 
 5   serving_size                             210691 non-null  object 
 6   serving_quantity                         210616 non-null  float64
 7   additives_n                              218451 non-null  float64
 8   ingredients_from_palm_oil_n              218451 non-null  float64
 9   ingredients_that_may_be_from_palm_oil_n  218451 non-null  float64
 10  nova_group                      

# SQL

In [7]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [9]:
products_df = clean_df # define data which has to be imported

# define database credentials
hostname="127.0.0.1"
dbname="nutrition"
uname="root"
pwd="root"

# define engine connection
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))
# importing data into database table
products_df.to_sql('products',con=engine,index=False,if_exists='append')
# connect to the database
engine.connect()

In [10]:
# dispose engine
engine.dispose()

## SQL Queries

* Nombre de produits par nutriscore.
  Quel est le pourcentage de représentation associé?

* Les 4 valeurs possibles du Nova_Group.

* Quel est la moyenne des fibres par nutriscore ?

* Quels sont le TOP10 des pnns_groups_2 qui comportent le plus d'additifs

* Afficher le taux de sucre/fat moyen et médian des produits avec un nutriscore A ou B

* Combien avons-nous de produits qui ont un taux de carbohydrates > aux sucres

In [14]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
# importing needed library
import mysql.connector

# establishing MySQL db connection
cnx = mysql.connector.connect(user='root', password='root', database='nutrition')
cursor = cnx.cursor()

In [4]:
# Nombre de produits par nutriscore
query_1 = ("SELECT count(*) AS 'products_count', nutriscore_grade FROM nutrition.products "
           "GROUP by nutriscore_grade;")

# stocking query results into pandas df
df_query_1 = pd.read_sql_query(query_1, cnx)
df_query_1

,products_count,nutriscore_grade
0,130657,0
1,54556,d
2,19746,b
3,31870,a
4,32357,c
5,30814,e


In [11]:
#  Les 4 valeurs possibles du Nova_Group
query_2 = ("SELECT nova_group FROM nutrition.products "
           "GROUP BY Nova_group;")

# stocking query results into pandas df
df_query_2 = pd.read_sql_query(query_2, cnx)
df_query_2

,nova_group
0,0.0
1,4.0
2,3.0
3,1.0
4,2.0


In [14]:
# Quel est la moyenne des fibres par nutriscore?
query_3 = ("SELECT AVG(fiber_100g) AS 'moyenne fibres', nutriscore_grade FROM nutrition.products "
           "GROUP BY nutriscore_grade;")

# stocking query results into pandas df
df_query_3 = pd.read_sql_query(query_3, cnx)
df_query_3

,moyenne fibres,nutriscore_grade
0,0.527872,0
1,1.815930,d
2,1.713789,b
3,4.076846,a
4,2.718985,c
5,1.414977,e


In [23]:
# Quels sont le TOP10 des pnns_groups_2 qui comportent le plus d'additifs
query_4 = ("SELECT pnns_groups_2, additives_n FROM nutrition.products "
           "ORDER BY pnns_groups_2 LIMIT 10")

# stocking query results into pandas df
df_query_4 = pd.read_sql_query(query_4, cnx)
df_query_4

,pnns_groups_2,additives_n
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
5,0,0.0
6,0,0.0
7,0,0.0
8,0,0.0
9,0,0.0


In [31]:
# Afficher le taux de sucre/fat moyen et médian des produits avec un nutriscore A ou B
query_5 = ("SELECT fat_100g, sugars_100g, AVG(*) FROM nutrition.products "
           "WHERE nutriscore_grade = 'A'")

# stocking query results into pandas df
df_query_5 = pd.read_sql_query(query_5, cnx)
df_query_5

DatabaseError: Execution failed on sql 'SELECT fat_100g, sugars_100g, AVG(*) FROM nutrition.products WHERE nutriscore_grade = 'A'': 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '*) FROM nutrition.products WHERE nutriscore_grade = 'A'' at line 1

In [ ]:
# Combien avons-nous de produits qui ont un taux de carbohydrates > aux sucres

In [ ]:
# closing cursor / connection
cursor.close()
cnx.close()